In [ ]:
# --- Imports ---
import numpy as np
import matplotlib.pyplot as plt
from numpy.linalg import norm
import networkx as nx
from time import time
from mpl_toolkits.mplot3d import Axes3D  # noqa: F401
from matplotlib import cm
from sklearn.linear_model import LinearRegression

from genet.estimators import GenElasticNetEstimator


# ----------------------- Utilities -----------------------

import numpy as np
import networkx as nx

import numpy as np
import networkx as nx
from itertools import product

def grid_graph_incidence(shape):
    """
    Build an axis-aligned lattice graph for a given shape (3D: (H, W), 3D: (A, B, C))
    and return:
      - D: edge-by-node oriented incidence matrix (m, p) with columns ordered in C-order
      - ravel: tensor -> vector in that same C-order
      - unravel: vector -> tensor with given shape
      - coords: list of node coordinate tuples in column order of D
    """
    # Normalize shape and build explicit coordinate ranges
    shape = tuple(int(s) for s in shape)
    assert 2 <= len(shape) <= 3, "shape must be 3D or 3D"

    ranges = [range(s) for s in shape]
    # In itertools.product(*ranges), the rightmost range varies fastest
    # This matches NumPy's C-order flattening.
    coords = list(product(*ranges))  # e.g., (i, j) or (i, j, k)

    # Build the lattice graph; keep tuple labels (no relabeling)
    try:
        G = nx.grid_graph(dim=ranges)   # works on many versions
    except TypeError:
        G = nx.grid_graph(ranges)       # fallback signature

    # By default, incidence_matrix returns (n_nodes, n_edges) — we transpose to (m, p)
    D = nx.incidence_matrix(G, oriented=True)
    D = D.T.astype(float).toarray()     # shape (m, p), edge-by-node

    # Ravel / unravel consistent with coords order (C-order)
    def ravel(tensor_beta):
        return np.asarray(tensor_beta, dtype=float).reshape(-1, order="C")

    def unravel(vec_beta):
        return np.asarray(vec_beta, dtype=float).reshape(shape, order="C")

    return D, ravel, unravel, coords




def beta_true_3D(shape, levels=(1.0, -0.8)):
    """Piecewise-smooth 2D beta (two rectangular blobs with smooth edges)."""
    H, W = shape
    B = np.zeros((H, W))
    B[H//8: 3*H//8, W//6: W//3] = levels[0]
    B[H//2: 7*H//8, 2*W//3: 5*W//6] = levels[1]
    # optional light smoothing (Gaussian-like via averaging)
    for _ in range(3):
        B = 0.25*(np.pad(B, 1, mode='edge')[2:,1:-1] + np.pad(B, 1, mode='edge')[:-2,1:-1] +
                  np.pad(B, 1, mode='edge')[1:-1,2:] + np.pad(B, 1, mode='edge')[1:-1,:-2])
    return B


def beta_true_3d(shape, level=1.0):
    """A single cuboid block inside a 3D grid."""
    A, B, C = shape
    V = np.zeros((A, B, C))
    V[A//4: A//2, B//4: 3*B//5, C//3: 2*C//3] = level
    return V


def fit_rmse_grid(X, y, Xte, yte, D, L1_GRID, L2_GRID, solver="admm"):
    """
    Sweep (l1,l2) and return:
       rmse_grid: [len(L1) x len(L2)]
       best: dict with l1,l2,rmse,beta_hat
    """
    rmse_grid = np.empty((len(L1_GRID), len(L2_GRID)))
    best = {"rmse": np.inf, "l1": None, "l2": None, "beta": None}

    for i, l1 in enumerate(L1_GRID):
        for j, l2 in enumerate(L2_GRID):
            est = GenElasticNetEstimator(l1=l1, l2=l2, D=D, family="normal", solver=solver)
            try:
                est.fit(X, y)
                yhat = est.predict(Xte)
                rmse = norm(yte - yhat) / np.sqrt(Xte.shape[0])
                rmse_grid[i, j] = rmse
                if rmse < best["rmse"]:
                    best.update({"rmse": rmse, "l1": l1, "l2": l2, "beta": est.beta.copy()})
            except Exception:
                rmse_grid[i, j] = np.nan

    return rmse_grid, best


def plot_rmse_heatmap(rmse_grid, L1_GRID, L2_GRID, title):
    plt.figure(figsize=(6.6, 5))
    im = plt.imshow(
        rmse_grid, origin="lower", aspect="auto", cmap="viridis",
        extent=[np.log10(L2_GRID[0]), np.log10(L2_GRID[-1]),
                np.log10(L1_GRID[0]), np.log10(L1_GRID[-1])]
    )
    plt.colorbar(im, label="RMSE")
    cs = plt.contour(np.log10(L2_GRID), np.log10(L1_GRID), rmse_grid, colors="w", linewidths=0.7)
    plt.clabel(cs, inline=True, fontsize=8, fmt="%.3f")
    plt.xlabel(r"$\log_{10}\,\lambda_2$")
    plt.ylabel(r"$\log_{10}\,\lambda_1$")
    plt.title(title)
    plt.tight_layout()
    plt.show()


def plot_beta_maps_2d(beta_true_grid, beta_hat_grid, title_prefix="2D"):
    diff = beta_hat_grid - beta_true_grid
    v = np.nanmax(np.abs(np.concatenate([beta_true_grid.ravel(), beta_hat_grid.ravel()])))
    fig, axs = plt.subplots(1, 3, figsize=(12, 3.6))
    im0 = axs[0].imshow(beta_true_grid, cmap="coolwarm", vmin=-v, vmax=v); axs[0].set_title(f"{title_prefix}: β★ (true)")
    im1 = axs[1].imshow(beta_hat_grid, cmap="coolwarm", vmin=-v, vmax=v); axs[1].set_title(f"{title_prefix}: β̂ (est)")
    im2 = axs[2].imshow(diff, cmap="bwr"); axs[2].set_title(f"{title_prefix}: β̂ − β★")
    for ax in axs: ax.axis("off")
    plt.colorbar(im0, ax=axs[0], fraction=0.046, pad=0.04)
    plt.colorbar(im1, ax=axs[1], fraction=0.046, pad=0.04)
    plt.colorbar(im2, ax=axs[2], fraction=0.046, pad=0.04)
    plt.tight_layout()
    plt.show()
    
# ----------------------- 2D experiment -----------------------

# Grid shape & truth
# Grid shape & truth
shape3d = (5, 5, 5)  # A x B x C 
D3, ravel3, unravel3, _ = grid_graph_incidence(shape3d)
V3_true = beta_true_3d(shape3d, level=1.0)
beta3_true = ravel3(V3_true)

# Synthetic data
rng = np.random.default_rng(123)
n = 300
p3 = beta3_true.size
X3 = rng.standard_normal((n, p3))
y3 = X3 @ beta3_true +  rng.standard_normal(n)
X3_te = rng.standard_normal((n, p3))
y3_te = X3_te @ beta3_true + rng.standard_normal(n)

# OLS baseline
ols2 = LinearRegression().fit(X3, y3)
rmse3_ols = norm(y3_te - ols2.predict(X3_te)) / np.sqrt(n)
print(f"[3D] OLS RMSE: {rmse3_ols:.4f}")

# GEN sweep
L1_GRID_3D= np.logspace(-2, 1, 20)   # 0.01 .. 10
L2_GRID_3D= np.logspace(-3, 0.5, 5) # 0.001 .. ~3.16
t0 = time()
rmse3_grid, best3 = fit_rmse_grid(X3, y3, X3_te, y3_te, D3, L1_GRID_3D, L2_GRID_3D, solver="admm")
print(f"[3D] Best RMSE (ADMM): {best3['rmse']:.4f} at (l1={best3['l1']:.3g}, l2={best3['l2']:.3g}); grid time: {time()-t0:.1f}s")

# Visuals: heatmap + 3D surface + maps
plot_rmse_heatmap(rmse3_grid, L1_GRID_3D, L2_GRID_3D, title=f"3D RMSE grid (OLS={rmse3_ols:.3f})")




[3D] OLS RMSE: 1.3784


In [7]:
len(L1_GRID_3D)

50

In [ ]:

t0 = time()
rmse3_grid, best3_cgd = fit_rmse_grid(X3, y3, X3_te, y3_te, D2, L1_GRID_3D, L2_GRID_3D, solver="cgd")
print(f"[2D] Best RMSE (CGD): {best3_cgd['rmse']:.4f} at (l1={best3_cgd['l1']:.3g}, l2={best3_cgd['l2']:.3g}); grid time: {time()-t0:.1f}s")

# Visuals: heatmap + 3D surface + maps
plot_rmse_heatmap(rmse3_grid, L1_GRID_3D, L2_GRID_3D, title=f"2D RMSE grid (OLS={rmse3_ols:.3f})")



t0 = time()
rmse3_grid, best3_cvx = fit_rmse_grid(X3, y3, X3_te, y3_te, D2, L1_GRID_3D, L2_GRID_3D, solver="cvxpy")
print(f"[2D] Best RMSE (CVX): {best3_cvx['rmse']:.4f} at (l1={best3_cvx['l1']:.3g}, l2={best3_cvx['l2']:.3g}); grid time: {time()-t0:.1f}s")

# Visuals: heatmap + 3D surface + maps
plot_rmse_heatmap(rmse3_grid, L1_GRID_3D, L2_GRID_3D, title=f"2D RMSE grid (OLS={rmse3_ols:.3f})")

t0 = time()
rmse3_grid, best3_ip = fit_rmse_grid(X3, y3, X3_te, y3_te, D2, L1_GRID_3D, L2_GRID_3D, solver="ip")
print(f"[2D] Best RMSE (CVX): {best3_ip['rmse']:.4f} at (l1={best3_ip['l1']:.3g}, l2={best3_ip['l2']:.3g}); grid time: {time()-t0:.1f}s")

# Visuals: heatmap + 3D surface + maps
plot_rmse_heatmap(rmse3_grid, L1_GRID_3D, L2_GRID_3D, title=f"2D RMSE grid (OLS={rmse3_ols:.3f})")



In [ ]:
# Visuals: heatmap + 3D surface + slices
plot_rmse_heatmap(rmse3_grid, L1_GRID_3D, L2_GRID_3D, title=f"3D RMSE grid (OLS={rmse3_ols:.3f})")
plot_rmse_3d_surface(rmse3_grid, L1_GRID_3D, L2_GRID_3D, title="3D RMSE surface")

V3_hat = unravel3(best3["beta"])
plot_beta_slices_3d(V3_true, V3_hat, title_prefix="3D grid")